# Task 2: Lookalike Model
## Build a Lookalike Model that takes a user's information as input and recommends 3 similar customers based on their profile and transaction history. The model should:
### ● Use both customer and product information.
### ● Assign a similarity score to each recommended customer.

In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [3]:

# Load the datasets
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

In [4]:
# Merge datasets for comprehensive analysis
df = pd.merge(transactions, customers, on='CustomerID')
df = pd.merge(df, products, on='ProductID')

In [5]:
# Prepare data for lookalike model
customer_features = pd.merge(customers, transactions.groupby('CustomerID').agg({'TotalValue': 'sum', 'Quantity': 'sum'}), on='CustomerID')
customer_features = pd.merge(customer_features, df.groupby('CustomerID')['Category'].agg(lambda x: x.mode()[0]), on='CustomerID')

In [6]:
# One-hot encode categorical features
customer_features = pd.get_dummies(customer_features, columns=['Region', 'Category'])

In [7]:
# Standardize features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features.drop(columns=['CustomerID', 'CustomerName', 'SignupDate']))

In [8]:
# Calculate cosine similarity
similarity_matrix = cosine_similarity(scaled_features)

In [9]:
# Function to get top 3 lookalikes
def get_lookalikes(customer_id, similarity_matrix, customer_features, top_n=3):
    customer_index = customer_features[customer_features['CustomerID'] == customer_id].index[0]
    similarity_scores = list(enumerate(similarity_matrix[customer_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_lookalikes = similarity_scores[1:top_n+1]  # Exclude the customer itself
    return [(customer_features.iloc[i[0]]['CustomerID'], i[1]) for i in top_lookalikes]

In [10]:
# Generate lookalikes for the first 20 customers
lookalike_map = {}
for customer_id in customer_features['CustomerID'].head(20):
    lookalike_map[customer_id] = get_lookalikes(customer_id, similarity_matrix, customer_features)

In [12]:
# Save to CSV
lookalike_df = pd.DataFrame([(k, v) for k, v in lookalike_map.items()], columns=['CustomerID', 'Lookalikes'])
lookalike_df.to_csv('HIMANSHU_LODHA_Lookalike.csv', index=False)